# Webscrapped City information to GCP SQL instance

---
## Import libraries 💾
If you haven't already installed sqlalchemy and pymysql, you will need to. Uncomment the code below, install, and then recomment the code - you only need to install it once.

In [69]:
# install if needed
# !pip install sqlalchemy
# !pip install pymysql

## Initialize connection string

In [12]:
from dotenv import load_dotenv
import os

load_dotenv("variables.env")

schema = os.getenv("schema")
host = os.getenv("host")
user = os.getenv("user")
password = os.getenv("password")
port = os.getenv("port")

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## Test your connection

In [36]:
from sqlalchemy import create_engine, text

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# Test the connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print(":white_check_mark: Connection successful:", result.scalar())
except Exception as e:
    print(":x: Connection failed:", e)

:white_check_mark: Connection successful: 1


## Webscrapping functions to get city information

In [16]:
import pandas as pd
from city_functions import CityPop, CityGeoInfo
Cities_pop_df = CityPop(['Berlin', 'Hamburg', 'Munich'])
Cities_Geo_df = CityGeoInfo(['Berlin', 'Hamburg', 'Munich'])

## Transfer city infos to data base

In [18]:
# transfer city names to sql data base
Cities_df = Cities_pop_df['City'].to_frame()
Cities_df.to_sql('city_names',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [20]:
# get city ids from sql data base
cities_from_sql = pd.read_sql("city_names", con=connection_string)

In [22]:
# transfer city population
# merge with cities_from_sql to get city ids
cities_pop_to_sql_df = Cities_pop_df.merge(cities_from_sql,
                                   on = "City",
                                   how="left")

# drop unnecessary columns
cities_pop_to_sql_df = cities_pop_to_sql_df.drop(columns=["City"])

# transfer city poulation data to sql data base
cities_pop_to_sql_df.to_sql('population',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [30]:
# transfer city geo
# merge with cities_from_sql to get city ids
cities_geo_to_sql_df = Cities_Geo_df.merge(cities_from_sql,
                                   on = "City",
                                   how="left")

# drop unnecessary columns
cities_geo_to_sql_df = cities_geo_to_sql_df.drop(columns=['City', 'Country'])

# transfer city geo data to sql data base
cities_geo_to_sql_df.to_sql('geo',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [52]:
# transfer city airport icaos
# merge with cities_from_sql to get city ids
cities_icaos = pd.DataFrame({'City': ['Berlin', 'Hamburg', 'Munich'], 'icao': ['EDDB', 'EDDH', 'EDDM']})
cities_icao_to_sql_df = cities_icaos.merge(cities_from_sql,
                                   on = "City",
                                   how="left")

# drop unnecessary columns
cities_icao_to_sql_df = cities_icao_to_sql_df.drop(columns=["City"])

# transfer city poulation data to sql data base
cities_icao_to_sql_df.to_sql('airport',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [60]:
icaos_df = pd.read_sql("airport", con=connection_string)
icaos_list = list(icaos_df['icao'])
icaos_list

['EDDB', 'EDDH', 'EDDM']